In [75]:
import torch
import matplotlib.pyplot as plt
import torch.nn.functional as F
%matplotlib inline

words = open("60knames.csv",'r').read().splitlines()[1:]
words = [word for word in words if not any(char in word for char in ['/','"',',','-',".","/",";",'±','Ã'])]

stoi = {c:i+1 for i,c in enumerate(sorted(set("".join(words))))}
stoi['.'] = 0
itos = {v:k for k,v in stoi.items()}

context_size = 3
words[:15]

inp = []
out = []
for word in words[:1]:
    context = context_size * '.'
    for ch in word:
        int_inp = [stoi[ch] for ch in context]
        inp.append(int_inp)
        out.append(stoi[ch])
        print(f'{context} -> {ch}')
        context = context[1:] + ch
print(inp, out)

inp = torch.tensor(inp)
out = torch.tensor(out)
inp, out

... -> A
..A -> A
.AA -> B
AAB -> A
ABA -> N
[[0, 0, 0], [0, 0, 1], [0, 1, 1], [1, 1, 2], [1, 2, 1]] [1, 1, 2, 1, 14]


(tensor([[0, 0, 0],
         [0, 0, 1],
         [0, 1, 1],
         [1, 1, 2],
         [1, 2, 1]]),
 tensor([ 1,  1,  2,  1, 14]))

In [111]:
g = torch.Generator().manual_seed(16)
C = torch.randn((27,2),generator = g)
emb = C[inp]

# emb_squeze = torch.cat((emb[:,0,:],emb[:,1,:],emb[:,2,:]),1)
# emb_squeze = torch.cat(torch.unbind(emb,1),1)
emb_squeze = emb.view(-1,6)


W = torch.randn((6,100),generator = g)
b = torch.randn((100),generator = g)
h = torch.tanh(emb_squeze @ W + b)
emb.shape, emb_squeze.shape, h

(torch.Size([5, 3, 2]),
 torch.Size([5, 6]),
 tensor([[-0.9095,  0.5782,  0.8243, -0.9278, -0.2169, -0.5027, -0.4617,  0.9991,
          -0.4904, -0.2298,  0.6110,  0.7172,  0.9668, -0.5528,  0.9209,  0.8414,
          -0.1187, -0.9910,  0.9673, -0.9964, -0.9781, -0.7835,  0.4021,  0.2669,
           0.9754,  0.9989, -0.1245, -0.9987,  0.7768, -0.9166, -0.9999, -0.5812,
          -0.0437,  0.9243,  0.5491, -0.8496, -0.9385,  0.3303,  0.7488, -0.9222,
           0.9720, -0.8548, -0.0279, -0.1609,  0.9990, -0.8807,  0.8811,  0.8228,
          -0.7067,  0.9977,  0.5092,  0.9934, -0.1448,  0.2306,  0.5047,  0.9983,
           0.2317,  0.9373, -0.3136, -1.0000,  0.9842,  0.6102,  0.9675, -0.9954,
           0.6975, -0.9895, -0.2316,  0.0304, -0.4112, -0.9441,  0.9569,  0.9129,
          -0.8603,  0.9754, -0.2191,  0.6048, -0.9844, -0.9845, -0.9562, -0.2188,
          -0.3092, -0.0445,  0.9313, -0.8103,  0.9986,  0.5198,  0.7998,  0.4435,
           0.9659,  0.8538, -0.4911,  0.9902, -0.9101

In [170]:
g = torch.Generator().manual_seed(16)
W2 = torch.randn((100,27),generator = g)
b2 = torch.randn((27),generator = g)

logits = h @ W2 + b2

# counts = logits.exp()
# p_counts = counts/counts.sum(1,keepdim= True)
# loss = -p_counts[torch.arange(5),out].log().mean()

loss = F.cross_entropy(logits,out)
loss

tensor(22.3540)